In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
from math import *

def model_inputs(real_dim, z_dim):
    inputs_real = tf.placeholder(tf.float32, (None, real_dim, real_dim, 1), name="inputs_real")
    inputs_z = tf.placeholder(tf.float32, (None, z_dim))
    return inputs_real, inputs_z

def generator(z, input_size=64, reuse=False,  alpha=0.01):
    with tf.variable_scope('generator', reuse=reuse):
        # Hidden layer 1
        h1 = tf.layers.dense(z, (input_size//8)*(input_size//8)*256, activation=None)
        h1 = tf.reshape(h1, [-1, (input_size//8), (input_size//8), 256])
        h1 = tf.maximum(h1, alpha*h1)
        h1 = tf.nn.dropout(h1, 0.4)
        # Upsampling-convolution layer 1
        upsample1 = tf.image.resize_nearest_neighbor(h1, [(input_size//4), (input_size//4)])
        conv1_1 = tf.layers.conv2d(inputs=upsample1, filters=256, kernel_size=[5, 5], padding="same", activation=None, name = "conv1_1")
        conv1_1 = tf.maximum(conv1_1, alpha*conv1_1)
        norm1_1 = tf.contrib.layers.batch_norm(inputs = conv1_1)
        conv1_2 = tf.layers.conv2d(inputs=norm1_1, filters=256, kernel_size=[5, 5], padding="same", activation=None, name = "conv1_2")
        conv1_2 = tf.maximum(conv1_2, alpha*conv1_2)
        norm1_2 = tf.contrib.layers.batch_norm(inputs = conv1_2)
        conv1_3 = tf.layers.conv2d(inputs=norm1_2, filters=256, kernel_size=[5, 5], padding="same", activation=None, name = "conv1_3")
        conv1_3 = tf.maximum(conv1_3, alpha*conv1_3)
        norm1_3 = tf.contrib.layers.batch_norm(inputs = conv1_3)
        conv1_4 = tf.layers.conv2d(inputs=norm1_3, filters=256, kernel_size=[5, 5], padding="same", activation=None, name = "conv1_4")
        conv1_4 = tf.maximum(conv1_4, alpha*conv1_4)
        norm1_4 = tf.contrib.layers.batch_norm(inputs = conv1_4)
        
        # Upsampling-convolution layer 2
        upsample2 = tf.image.resize_nearest_neighbor(norm1_4, [input_size//2, input_size//2])
        conv2_1 = tf.layers.conv2d(inputs=upsample2, filters=128, kernel_size=[5, 5], padding="same", activation=None, name = "conv2_1")
        conv2_1 = tf.maximum(conv2_1, alpha*conv2_1)
        norm2_1 = tf.contrib.layers.batch_norm(inputs = conv2_1)
        conv2_2 = tf.layers.conv2d(inputs=norm2_1, filters=128, kernel_size=[5, 5], padding="same", activation=None, name = "conv2_2")
        conv2_2 = tf.maximum(conv2_2, alpha*conv2_2)
        norm2_2 = tf.contrib.layers.batch_norm(inputs = conv2_2)
        conv2_3 = tf.layers.conv2d(inputs=norm2_2, filters=128, kernel_size=[5, 5], padding="same", activation=None, name = "conv2_3")
        conv2_3 = tf.maximum(conv2_3, alpha*conv2_3)
        norm2_3 = tf.contrib.layers.batch_norm(inputs = conv2_3)
        conv2_4 = tf.layers.conv2d(inputs=norm2_3, filters=128, kernel_size=[5, 5], padding="same", activation=None, name = "conv2_4")
        conv2_4 = tf.maximum(conv2_4, alpha*conv2_4)
        norm2_4 = tf.contrib.layers.batch_norm(inputs = conv2_4)
        
        # Upsampling-convolution layer 3
        upsample3 = tf.image.resize_nearest_neighbor(norm2_4, [input_size, input_size])
        conv3_1 = tf.layers.conv2d(inputs=upsample3, filters=64, kernel_size=[5, 5], padding="same", activation=None, name = "conv3_1")
        conv3_1 = tf.maximum(conv3_1, alpha*conv3_1)
        norm3_1 = tf.contrib.layers.batch_norm(inputs = conv3_1)
        conv3_2 = tf.layers.conv2d(inputs=norm3_1, filters=64, kernel_size=[5, 5], padding="same", activation=None, name = "conv3_2")
        conv3_2 = tf.maximum(conv3_2, alpha*conv3_2)
        norm3_2 = tf.contrib.layers.batch_norm(inputs = conv3_2)
        conv3_3 = tf.layers.conv2d(inputs=norm3_2, filters=64, kernel_size=[5, 5], padding="same", activation=None, name = "conv3_3")
        conv3_3 = tf.maximum(conv3_3, alpha*conv3_3)
        norm3_3 = tf.contrib.layers.batch_norm(inputs = conv3_3)
        conv3_4 = tf.layers.conv2d(inputs=norm3_3, filters=1, kernel_size=[5, 5], padding="same", activation=None, name = "conv3_4")
        
        # Logits and tanh output
        logits = conv3_4
        out = tf.nn.tanh(logits)
        tf.summary.image("Generated_image", (tf.reshape(out, [-1, input_size, input_size, 1])+1) * 0.5, 3)
        
        return out, logits
        
def discriminator(x, input_size=28, reuse=False, alpha=0.01):
    with tf.variable_scope('discriminator', reuse=reuse):
        # Convolutional layer 1
        conv1_1 = tf.layers.conv2d(inputs=x, filters=32, kernel_size=[5, 5], padding="same", activation=None, name = "conv1_1")
        conv1_1 = tf.maximum(conv1_1, alpha*conv1_1)
        norm1_1 = tf.contrib.layers.batch_norm(inputs = conv1_1)
        conv1_2 = tf.layers.conv2d(inputs=norm1_1, filters=32, kernel_size=[5, 5], padding="same", activation=None, name = "conv1_2")
        conv1_2 = tf.maximum(conv1_2, alpha*conv1_2)
        norm1_2 = tf.contrib.layers.batch_norm(inputs = conv1_2)
        pool1 = tf.layers.max_pooling2d(inputs=norm1_2, pool_size=[2, 2], strides=2, name = "pool1")
        # Convolutional layer 2
        conv2_1 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5, 5], padding="same", activation=None, name = "conv2_1")
        conv2_1 = tf.maximum(conv2_1, alpha*conv2_1)
        norm2_1 = tf.contrib.layers.batch_norm(inputs = conv2_1)
        conv2_2 = tf.layers.conv2d(inputs=norm2_1, filters=64, kernel_size=[5, 5], padding="same", activation=None, name = "conv2_2")
        conv2_2 = tf.maximum(conv2_2, alpha*conv2_2)
        norm2_2 = tf.contrib.layers.batch_norm(inputs = conv2_2)
        pool2 = tf.layers.max_pooling2d(inputs=norm2_2, pool_size=[2, 2], strides=2, name = "pool2")
        # Convolutional layer 2
        conv3_1 = tf.layers.conv2d(inputs=pool2, filters=128, kernel_size=[5, 5], padding="same", activation=None, name = "conv3_1")
        conv3_1 = tf.maximum(conv3_1, alpha*conv3_1)
        norm3_1 = tf.contrib.layers.batch_norm(inputs = conv3_1)
        conv3_2 = tf.layers.conv2d(inputs=norm3_1, filters=128, kernel_size=[5, 5], padding="same", activation=None, name = "conv3_2")
        conv3_2 = tf.maximum(conv3_2, alpha*conv3_2)
        norm3_2 = tf.contrib.layers.batch_norm(inputs = conv3_2)
        pool3 = tf.layers.max_pooling2d(inputs=norm3_2, pool_size=[2, 2], strides=2, name = "pool3")
        flatten = tf.reshape(pool3, [-1, 128*(input_size//8)*(input_size//8)])
        flatten = tf.nn.dropout(flatten, 0.4)
        logits = tf.layers.dense(flatten, 1, activation=None)
        out = tf.nn.sigmoid(logits)
        
        return out, logits
        
# Size of input image to discriminator
input_size = 64 #Side of the image
# Size of latent vector to generator
z_size = 100
# Leak factor for leaky ReLU
alpha = 0.01
# Label smoothing 
smooth = 0.1

tf.reset_default_graph()
# Create our input placeholders
input_real, input_z = model_inputs(input_size, z_size)
tf.summary.image("Real_image_sample", (tf.reshape(input_real, [-1, input_size, input_size, 1])+1) * 0.5, 3)
# Generator network here
g_model, g_logits = generator(input_z, input_size, reuse=False,  alpha=alpha)
# g_model is the generator output

# Disriminator network here
d_model_real, d_logits_real = discriminator(input_real, input_size, reuse=False, alpha=alpha)
d_model_fake, d_logits_fake = discriminator(g_model, input_size, reuse=True, alpha=alpha)

# Calculate losses
d_labels_real = tf.ones_like(d_logits_real) * (1 - smooth)
d_labels_fake = tf.zeros_like(d_logits_fake)

d_loss_real = tf.nn.sigmoid_cross_entropy_with_logits(labels=d_labels_real, logits=d_logits_real)
d_loss_fake = tf.nn.sigmoid_cross_entropy_with_logits(labels=d_labels_fake, logits=d_logits_fake)

d_loss = tf.reduce_mean(d_loss_real + d_loss_fake)
tf.summary.scalar("Disc_loss", d_loss)
g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
        labels=tf.ones_like(d_logits_fake), 
        logits=d_logits_fake))
tf.summary.scalar("Gen_loss", g_loss)
tf.summary.scalar("Mean_difference_disc", tf.reduce_mean(tf.ones_like(d_logits_fake)-d_model_fake))
# Optimizers
disc_lr = 0.001
gen_lr = 0.0001

# Get the trainable_variables, split into G and D parts
t_vars = tf.trainable_variables()
g_vars = [var for var in t_vars if var.name.startswith("generator")]
d_vars = [var for var in t_vars if var.name.startswith("discriminator")]

d_train_opt = tf.train.AdamOptimizer(learning_rate=disc_lr).minimize(d_loss, var_list=d_vars)
g_train_opt = tf.train.AdamOptimizer(learning_rate=gen_lr).minimize(g_loss, var_list=g_vars)

logging_step = 50
data = np.load('Datasets/olivetti_faces.npy')
batch_size = 32
epochs = 1000
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, 'Weights/ganFaces1')
    merged_summary = tf.summary.merge_all()
    summary_counter = 0
    writer = tf.summary.FileWriter("./Logs")
    writer.add_graph(sess.graph)
    for e in range(epochs):
        np.random.shuffle(data)
        for ii in range(data.shape[0]//batch_size):
            batch = data[ii*batch_size:(ii+1)*batch_size]
            
            # Get images, reshape and rescale to pass to D
            batch_images = batch.reshape((batch_size, input_size, input_size, 1))
            batch_images = batch_images*2 - 1
            
            # Sample random noise for G
            batch_z = np.random.uniform(-1, 1, size=(batch_size, z_size))
            
            # Run optimizers
            sess.run(d_train_opt, feed_dict={input_real: batch_images, input_z: batch_z})
            sess.run(g_train_opt, feed_dict={input_z: batch_z})
            #Logging
            if ii % logging_step == 0:
                s = sess.run(merged_summary, feed_dict={input_real: batch_images, input_z: batch_z})
                writer.add_summary(s, summary_counter*logging_step)
                summary_counter += 1
        
        # At the end of each epoch, get the losses and print them out
        train_loss_d = sess.run(d_loss, {input_z: batch_z, input_real: batch_images})
        train_loss_g = g_loss.eval({input_z: batch_z})
            
        print("Epoch {}/{}...".format(e+1, epochs),
              "Discriminator Loss: {:.4f}...".format(train_loss_d),
              "Generator Loss: {:.4f}".format(train_loss_g))    
        saver.save(sess, 'Weights/ganFaces1_2')


